In [1]:
import pandas as pd
import numpy as np
import requests
import scipy as sp
import xlsxwriter
import warnings as w
import math
w.filterwarnings('ignore')

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 102488470, 'calculationPrice': 'tops', 'change': -4.18, 'changePercent': -0.02859, 'close': 0, 'closeSource': 'iafcofil', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 144.16, 'iexAskSize': 502, 'iexBidPrice': 142, 'iexBidSize': 129, 'iexClose': 146, 'iexCloseTime': 1730994338325, 'iexLastUpdated': 1735810051180, 'iexMarketPercent': 0.018168297947313873, 'iexOpen': 145.25, 'iexOpenTime': 1665994971264, 'iexRealtimePrice': 144.5, 'iexRealtimeSize': 9, 'iexVolume': 542484, 'lastTradeTime': 1673738333649, 'latestPrice': 145.6, 'latestSource': 'IEX real time price', 'latestTime': '11:30:22 AM', 'latestUpdate': 1672011472778, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2333035544959, 'oddLotDelayedPrice

In [5]:
price= data['latestPrice']
mrkt_cap= data['marketCap']
mrkt_cap=mrkt_cap/1000000000000

In [6]:
col= ['Tickers','Stock Price','Market Capitalization','Number of Share to Buy']
final_dataframe=pd.DataFrame(columns=col)

In [7]:
final_dataframe.append(pd.Series(
    [symbol,price,mrkt_cap,'N/A'],
    index=col),
    ignore_index=True
)

,Tickers,Stock Price,Market Capitalization,Number of Share to Buy
0,AAPL,145.6,2.333036,N/A


In [8]:
final_dataframe=pd.DataFrame(columns=col)
for stock in stocks['Ticker'][:5]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe=final_dataframe.append(pd.Series(
        [stock,data['latestPrice'],data['marketCap'],'N/A'],
        index=col),
    ignore_index=True
    )
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

In [9]:
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Share to Buy
0,A,129.95,38403765815,N/A
1,AAL,12.28,8275458565,N/A
2,AAP,162.609,9867832895,N/A
3,AAPL,143.299,2285316102757,N/A
4,ABBV,146.5,249416656783,N/A


In [10]:
def spt(ln,n):
    for i in range(0,len(ln),n):
        yield ln[i:i+n]

In [11]:
symbol_groups=list(spt(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns =col)
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index =col), 
                                        ignore_index = True)
        
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Share to Buy
0,A,133.56,38843748669,N/A
1,AAL,12.51,8268834908,N/A
2,AAP,169.6,9890698300,N/A
3,AAPL,143.391,2370987358827,N/A
4,ABBV,144.3,252289016702,N/A
...,...,...,...,...
496,YUM,112.663,31365454451,N/A
497,ZBH,109.76,22831984068,N/A
498,ZBRA,282.34,14408744442,N/A
499,ZION,52.828,7890264682,N/A


In [12]:
portfolio_size=input("Enter your portfolio size:")
try:
    val=float(portfolio_size)
except ValueError:
    print("That's not a number \n")
    portfolio_size=input("Enter your portfolio size:")
    val=float(portfolio_size) 

In [13]:
position_size=float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Share to Buy']=math.floor(position_size/final_dataframe['Stock Price'][i])
final_dataframe

,Tickers,Stock Price,Market Capitalization,Number of Share to Buy
0,A,133.56,38843748669,3736
1,AAL,12.51,8268834908,39888
2,AAP,169.6,9890698300,2942
3,AAPL,143.391,2370987358827,3480
4,ABBV,144.3,252289016702,3458
...,...,...,...,...
496,YUM,112.663,31365454451,4429
497,ZBH,109.76,22831984068,4546
498,ZBRA,282.34,14408744442,1767
499,ZION,52.828,7890264682,9445


In [14]:
final_dataframe.to_csv('Equal_weight_S&P_500.csv')